In [1]:
import pandas as pd
df = pd.read_csv('stock_data.csv')
df.head()

,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1


In [2]:
import re
cleaned = []
for text in df['Text'].fillna(''):
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    cleaned_data = cleaned_text.strip()
    cleaned.append(cleaned_data)

In [3]:
cleaned[1]

'user AAP MOVIE 55 return for the FEAGEED indicator just 15 trades for the year AWESOME'

In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [5]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
tokens = [word_tokenize(x) for x in cleaned]

In [7]:
tokens[0]

['Kickers',
 'on',
 'my',
 'watchlist',
 'XIDE',
 'TIT',
 'SOQ',
 'PNK',
 'CPW',
 'BPZ',
 'AJ',
 'trade',
 'method',
 '1',
 'or',
 'method',
 '2',
 'see',
 'prev',
 'posts']

In [8]:
stop = set(stopwords.words('english'))
stop_token = []
for k in range(len(df['Text'])):
    p = [i for i in tokens[k] if i not in stop]
    stop_token.append(p)

In [9]:
stop_token[0]

['Kickers',
 'watchlist',
 'XIDE',
 'TIT',
 'SOQ',
 'PNK',
 'CPW',
 'BPZ',
 'AJ',
 'trade',
 'method',
 '1',
 'method',
 '2',
 'see',
 'prev',
 'posts']

In [10]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
stemedata = []
for message in stop_token:
    st = [ps.stem(word) for word in message]
    stemedata.append(st)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [13]:
sen_stem=[''.join(i) for i in stemedata]
x_vec=cv.fit_transform(sen_stem).toarray()

In [14]:
x_vec

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [15]:
from sklearn.naive_bayes import MultinomialNB

In [16]:
mb=MultinomialNB()

In [17]:
df['sentiment_label'] = df['Sentiment'].map({1: 'Positive', -1: 'Negative'})

In [18]:
df

,Text,Sentiment,sentiment_label
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1,Positive
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1,Positive
2,user I'd be afraid to short AMZN - they are lo...,1,Positive
3,MNTA Over 12.00,1,Positive
4,OI Over 21.37,1,Positive
...,...,...,...
5786,Industry body CII said #discoms are likely to ...,-1,Negative
5787,"#Gold prices slip below Rs 46,000 as #investor...",-1,Negative
5788,Workers at Bajaj Auto have agreed to a 10% wag...,1,Positive
5789,"#Sharemarket LIVE: Sensex off day’s high, up 6...",1,Positive


In [19]:
y=df['sentiment_label']

In [20]:
y.shape

(5791,)

In [21]:
mb.fit(x_vec,y)

MultinomialNB()

In [22]:
mb.predict([x_vec[2]])

array(['Positive'], dtype='<U8')

In [23]:
import pickle
with open ('stock_model.pkl','wb') as file:
  pickle.dump(mb,file)

In [24]:
with open ('stock_model.pkl','rb') as file:
  model=pickle.load(file)

In [25]:
sen_stem=[''.join(i) for i in stemedata]
x_vec=cv.fit(sen_stem)

In [26]:
with open ('cv.pkl','wb') as file:
  pickle.dump(x_vec,file)

In [27]:
with open ('cv.pkl','rb') as file:
  rv=pickle.load(file)